# Preprocessing Take 2
The other one is too sus I will try to start with sparse matrix

In [42]:
import pandas as pd
import numpy as np
import scipy
import nltk
from string import punctuation
import string
import re


from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load data

In [43]:
# Check location of the data
df1 = pd.read_csv("data/spam.csv", usecols=[0,1])
df1['Category'] = df1['Category'].replace({'ham': 0, 'spam': 1})

df2 = pd.read_csv("data/enronSpamSubset.csv", usecols=[1,0])
df3 = pd.read_csv("data/lingSpam.csv", usecols=[1,0])
#df4 = pd.read_csv("data/completeSpamAssassin.csv", usecols=[1,0])

print(df2.shape, df3.shape)
df2.columns = df1.columns
df3.columns = df1.columns
#df4.columns = df1.columns

#Removed SpamAssassin because it looks like ass
allData = pd.concat([df1, df2, df3],ignore_index = True)
allData = allData[pd.to_numeric(allData['Category'], errors='coerce').notna()]
allData = allData.dropna()
allData.iloc[:, 0] = allData.iloc[:, 0].astype(int)
allData = allData[allData['Category'].isin([0, 1])]



C:\Users\jonat\AppData\Local\Temp\ipykernel_11968\4071407081.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Category'] = df1['Category'].replace({'ham': 0, 'spam': 1})
C:\Users\jonat\AppData\Local\Temp\ipykernel_11968\4071407081.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("data/enronSpamSubset.csv", usecols=[1,0])
C:\Users\jonat\AppData\Local\Temp\ipykernel_11968\4071407081.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("data/lingSpam.csv", usecols=[1,0])


(16146, 2) (23705, 2)


In [44]:
print(allData['Message'][0])
print("Count of label:\n",allData['Category'].value_counts())

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Count of label:
 Category
0    11976
1     6180
Name: count, dtype: int64


need to fix that one spreadsheet

### Remove punctuation, lemmatize, etc

In [45]:
def remove_punc(text):
    nonP_text = "".join([char for char in text if char not in string.punctuation])
    return nonP_text

allData['no_punc'] = allData['Message'].apply(lambda x: remove_punc(x))
allData

,Category,Message,no_punc
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...
...,...,...,...
30407,0,the position will be available from late octo...,the position will be available from late octo...
34810,0,"""Subject: 30",Subject 30
34840,0,its not the best but will get you started if ...,its not the best but will get you started if ...
35892,0,cleanest list on the internet today ! only $ ...,cleanest list on the internet today only 69...


In [46]:
def tokenize(text):
    tokens = re.split("\W+", text)
    return tokens

allData['tokenized'] = allData['no_punc'].apply(lambda x: tokenize(x))
allData

,Category,Message,no_punc,tokenized
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t..."
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l..."
...,...,...,...,...
30407,0,the position will be available from late octo...,the position will be available from late octo...,"[, the, position, will, be, available, from, l..."
34810,0,"""Subject: 30",Subject 30,"[Subject, 30, ]"
34840,0,its not the best but will get you started if ...,its not the best but will get you started if ...,"[, its, not, the, best, but, will, get, you, s..."
35892,0,cleanest list on the internet today ! only $ ...,cleanest list on the internet today only 69...,"[, cleanest, list, on, the, internet, today, o..."


In [47]:
stopwords = nltk.corpus.stopwords.words("english")

def remove_stopwords(token):
    text = [word for word in token if word not in stopwords]# to remove all stopwords
    return text

allData['nonstop'] = allData['tokenized'].apply(lambda x: remove_stopwords(x))
allData

,Category,Message,no_punc,tokenized,nonstop
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around..."
...,...,...,...,...,...
30407,0,the position will be available from late octo...,the position will be available from late octo...,"[, the, position, will, be, available, from, l...","[, position, available, late, october, 1995, e..."
34810,0,"""Subject: 30",Subject 30,"[Subject, 30, ]","[Subject, 30, ]"
34840,0,its not the best but will get you started if ...,its not the best but will get you started if ...,"[, its, not, the, best, but, will, get, you, s...","[, best, get, started, dont, one, prices, quot..."
35892,0,cleanest list on the internet today ! only $ ...,cleanest list on the internet today only 69...,"[, cleanest, list, on, the, internet, today, o...","[, cleanest, list, internet, today, 69, dollar..."


In [48]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag


# From https://www.ibm.com/topics/stemming-lemmatization 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN
       
def lemmatize_email(words):
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return lemmatized_words

#allData['cleaned_msg'] = allData['stemmed'].apply(lambda x: lemmatizer(x))
allData['Lemmas'] = allData['nonstop'].apply(lambda x: lemmatize_email(x))
allData

,Category,Message,no_punc,tokenized,nonstop,Lemmas
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n...","[Go, jurong, point, crazy, Available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]","[U, dun, say, early, hor, U, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around...","[Nah, I, dont, think, go, usf, life, around, t..."
...,...,...,...,...,...,...
30407,0,the position will be available from late octo...,the position will be available from late octo...,"[, the, position, will, be, available, from, l...","[, position, available, late, october, 1995, e...","[, position, available, late, october, 1995, e..."
34810,0,"""Subject: 30",Subject 30,"[Subject, 30, ]","[Subject, 30, ]","[Subject, 30, ]"
34840,0,its not the best but will get you started if ...,its not the best but will get you started if ...,"[, its, not, the, best, but, will, get, you, s...","[, best, get, started, dont, one, prices, quot...","[, best, get, start, dont, one, price, quote, ..."
35892,0,cleanest list on the internet today ! only $ ...,cleanest list on the internet today only 69...,"[, cleanest, list, on, the, internet, today, o...","[, cleanest, list, internet, today, 69, dollar...","[, clean, list, internet, today, 69, dollar, p..."


In [49]:
def compress_numbers(words):
    # for word in words:
    #     try: 
    #         word = float(word)
    #         word = 69
    #     except: continue
    # return words
    counter = 0
    for word in words:
        try: 
            float(word)
            words[counter] = '69'
            counter+=1
            continue
        except: pass
        if word.isnumeric():
            words[counter] = '69'
        else:
            pass
        counter+=1
    return words

## Compressing Numbers
print(compress_numbers(allData['Lemmas'][2]))
allData['CompressedNums'] = allData['Lemmas'].apply(lambda x: compress_numbers(x))
allData

['Free', 'entry', '69', 'wkly', 'comp', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '69', 'Text', 'FA', '69', 'receive', 'entry', 'questionstd', 'txt', 'rateTCs', 'apply', '08452810075over18s']


,Category,Message,no_punc,tokenized,nonstop,Lemmas,CompressedNums
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[Go, until, jurong, point, crazy, Available, o...","[Go, jurong, point, crazy, Available, bugis, n...","[Go, jurong, point, crazy, Available, bugis, n...","[Go, jurong, point, crazy, Available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]","[Ok, lar, Joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, 2, wkly, comp, win, FA, Cup, fin...","[Free, entry, 69, wkly, comp, win, FA, Cup, fi...","[Free, entry, 69, wkly, comp, win, FA, Cup, fi..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[U, dun, say, so, early, hor, U, c, already, t...","[U, dun, say, early, hor, U, c, already, say]","[U, dun, say, early, hor, U, c, already, say]","[U, dun, say, early, hor, U, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[Nah, I, dont, think, he, goes, to, usf, he, l...","[Nah, I, dont, think, goes, usf, lives, around...","[Nah, I, dont, think, go, usf, life, around, t...","[Nah, I, dont, think, go, usf, life, around, t..."
...,...,...,...,...,...,...,...
30407,0,the position will be available from late octo...,the position will be available from late octo...,"[, the, position, will, be, available, from, l...","[, position, available, late, october, 1995, e...","[, position, available, late, october, 69, ear...","[, position, available, late, october, 69, ear..."
34810,0,"""Subject: 30",Subject 30,"[Subject, 30, ]","[Subject, 30, ]","[Subject, 69, ]","[Subject, 69, ]"
34840,0,its not the best but will get you started if ...,its not the best but will get you started if ...,"[, its, not, the, best, but, will, get, you, s...","[, best, get, started, dont, one, prices, quot...","[, best, get, start, dont, one, price, quote, ...","[, best, get, start, dont, one, price, quote, ..."
35892,0,cleanest list on the internet today ! only $ ...,cleanest list on the internet today only 69...,"[, cleanest, list, on, the, internet, today, o...","[, cleanest, list, internet, today, 69, dollar...","[, clean, list, internet, today, 69, dollar, p...","[, clean, list, internet, today, 69, dollar, p..."


### Keep words that show up more than once

In [50]:
cleanData = allData[['Category', 'CompressedNums']].copy()
#cleanData['CompressedNums'][2]
allData['Lemmas'][2]
print(compress_numbers(allData['Lemmas'][2]))

['Free', 'entry', '69', 'wkly', 'comp', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '69', 'Text', 'FA', '69', 'receive', 'entry', 'questionstd', 'txt', 'rateTCs', 'apply', '08452810075over18s']


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

# Convert 'CompressedNums' to a single string per row
cleanData['CompressedNums'] = cleanData['CompressedNums'].apply(' '.join)

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'CompressedNums' column
X = vectorizer.fit_transform(cleanData['CompressedNums'])

# Convert the result to a sparse DataFrame
count_df = pd.DataFrame.sparse.from_spmatrix(X, columns=vectorizer.get_feature_names_out())

# Find words that appear in multiple emails
counts = count_df.sum(axis=0)
multiple_appearances = counts[counts >= 2].index

# Keep only the columns for words that appear in multiple emails
count_df = count_df[multiple_appearances]

# Concatenate the sparse DataFrame with the original DataFrame
cleanData = cleanData.reset_index(drop=True)
count_df = count_df.reset_index(drop=True)
cleanData = pd.concat([cleanData, count_df], axis=1)

In [52]:
print(cleanData.shape)
print(count_df.head())
print(multiple_appearances)
print(cleanData)

(18156, 56216)
   000yen  00a  00am  00coffee  00h  00hfstahlke  00j  00m  00p  00pm  ...  \
0       0    0     0         0    0            0    0    0    0     0  ...   
1       0    0     0         0    0            0    0    0    0     0  ...   
2       0    0     0         0    0            0    0    0    0     0  ...   
3       0    0     0         0    0            0    0    0    0     0  ...   
4       0    0     0         0    0            0    0    0    0     0  ...   

   útil  über  üll  ýle  ýngýlýzce  ýþ  ýþçý  þcómo  þãõ  þù  
0     0     0    0    0          0   0     0      0    0   0  
1     0     0    0    0          0   0     0      0    0   0  
2     0     0    0    0          0   0     0      0    0   0  
3     0     0    0    0          0   0     0      0    0   0  
4     0     0    0    0          0   0     0      0    0   0  

[5 rows x 56214 columns]
Index(['000yen', '00a', '00am', '00coffee', '00h', '00hfstahlke', '00j', '00m',
       '00p', '00pm',
       ...

In [53]:
from scipy.sparse import vstack

# Convert categorical data to numeric codes
for col in cleanData.columns:
    if cleanData[col].dtype == 'object':
        cleanData[col] = pd.Categorical(cleanData[col]).codes

# Convert each column to a sparse array and stack them
sparse_matrix = vstack([csr_matrix(cleanData[col].values) for col in cleanData.columns])
sparse_matrix = sparse_matrix.transpose()

# Keep a separate list of column names
column_names = cleanData.columns.tolist()



### Proof that it works with one email

In [54]:
print(sparse_matrix.shape)
print(sparse_matrix)

print(len(column_names))
print(column_names)

(18156, 56216)
  (2, 0)	1
  (5, 0)	1
  (8, 0)	1
  (9, 0)	1
  (11, 0)	1
  (12, 0)	1
  (15, 0)	1
  (19, 0)	1
  (34, 0)	1
  (42, 0)	1
  (54, 0)	1
  (56, 0)	1
  (65, 0)	1
  (67, 0)	1
  (68, 0)	1
  (93, 0)	1
  (95, 0)	1
  (114, 0)	1
  (117, 0)	1
  (120, 0)	1
  (121, 0)	1
  (123, 0)	1
  (134, 0)	1
  (135, 0)	1
  (139, 0)	1
  :	:
  (8386, 56202)	2
  (8669, 56203)	1
  (10316, 56203)	1
  (8805, 56204)	1
  (9218, 56204)	1
  (9565, 56205)	1
  (9814, 56205)	1
  (6232, 56206)	1
  (9353, 56206)	1
  (5661, 56207)	9
  (6860, 56207)	1
  (8698, 56207)	1
  (125, 56208)	1
  (1228, 56208)	1
  (4422, 56208)	1
  (5648, 56209)	1
  (7464, 56209)	1
  (5648, 56210)	2
  (6025, 56211)	2
  (6025, 56212)	2
  (6151, 56213)	2
  (8669, 56214)	1
  (10316, 56214)	1
  (6340, 56215)	1
  (8696, 56215)	1
56216
['Category', 'CompressedNums', '000yen', '00a', '00am', '00coffee', '00h', '00hfstahlke', '00j', '00m', '00p', '00pm', '00tea', '013a', '01bd8356', '01in', '0776xxxxxxx', '07xxxxxxxxx', '08452810075over18s', '087006211

In [55]:
# Get the indices of non-zero elements in the first row of the sparse matrix
non_zero_indices = sparse_matrix[1].nonzero()[1]

# Get the corresponding column names
tokens = [column_names[i] for i in non_zero_indices]
print(tokens)

['CompressedNums', 'joking', 'lar', 'ok', 'oni', 'wif']


### Save output

In [56]:
scipy.sparse.save_npz("data/sparse_df.npz", sparse_matrix)
# Save column names to a file
with open('data/column_names.txt', 'w') as f:
    for item in column_names:
        f.write("%s\n" % item)


In [58]:
print(sparse_matrix[:,1])

  (0, 0)	1060
  (1, 0)	3125
  (2, 0)	981
  (3, 0)	16024
  (4, 0)	2757
  (5, 0)	992
  (6, 0)	889
  (7, 0)	340
  (8, 0)	16178
  (9, 0)	1218
  (10, 0)	2151
  (11, 0)	3414
  (12, 0)	16079
  (13, 0)	2330
  (14, 0)	1680
  (15, 0)	16520
  (16, 0)	3029
  (17, 0)	866
  (18, 0)	945
  (19, 0)	873
  (20, 0)	2243
  (21, 0)	1800
  (22, 0)	3592
  (23, 0)	123
  (24, 0)	936
  :	:
  (18131, 0)	4979
  (18132, 0)	4982
  (18133, 0)	4975
  (18134, 0)	4978
  (18135, 0)	6198
  (18136, 0)	5616
  (18137, 0)	14580
  (18138, 0)	5721
  (18139, 0)	14417
  (18140, 0)	4671
  (18141, 0)	5727
  (18142, 0)	15505
  (18143, 0)	5034
  (18144, 0)	5836
  (18145, 0)	4998
  (18146, 0)	15277
  (18147, 0)	7191
  (18148, 0)	3746
  (18149, 0)	14
  (18150, 0)	15
  (18151, 0)	22
  (18152, 0)	3746
  (18153, 0)	14
  (18154, 0)	15
  (18155, 0)	22
